In [ ]:
# Backtesting the Arbitrage Strategy
def backtest_arbitrage(df):
    initial_balance = 1000  # Start with $1000
    balance = initial_balance
    position = 0  # No initial position
    transaction_history = []

    for i in range(1, len(df)):
        spot_price = df.loc[i, 'close_spot']
        um_futures_price = df.loc[i, 'close_um']
        cm_futures_price = df.loc[i, 'close_cm']

        # Calculate arbitrage opportunities
        um_arbitrage_profit = um_futures_price - spot_price
        cm_arbitrage_profit = cm_futures_price - spot_price

        # Simulate arbitrage trades
        if um_arbitrage_profit > fee_rate:  # If there's a profit in UM arbitrage
            if position == 0:  # No active position
                position = trade_size  # Buy on spot market
                balance -= spot_price * trade_size  # Deduct the capital
                transaction_history.append(('buy', 'spot', spot_price, trade_size))

            # Sell on Unified Margin Futures
            balance += um_futures_price * trade_size - (um_futures_price * trade_size * fee_rate)  # Deduct fees
            position = 0  # Close the position
            transaction_history.append(('sell', 'UM Futures', um_futures_price, trade_size))

        elif cm_arbitrage_profit > fee_rate:  # If there's a profit in CM arbitrage
            if position == 0:  # No active position
                position = trade_size  # Buy on spot market
                balance -= spot_price * trade_size  # Deduct the capital
                transaction_history.append(('buy', 'spot', spot_price, trade_size))

            # Sell on Coin-Margin Futures
            balance += cm_futures_price * trade_size - (cm_futures_price * trade_size * fee_rate)  # Deduct fees
            position = 0  # Close the position
            transaction_history.append(('sell', 'CM Futures', cm_futures_price, trade_size))

        # Output the balance after each trade (for tracking)
        print(f"Timestamp: {df.loc[i, 'timestamp']}, Balance: {balance:.2f}, Position: {position}")

    # Return the final balance and transaction history
    return balance, transaction_history

# Plotting the results
def plot_results(transaction_history):
    # Convert the transaction history into a DataFrame for easy plotting
    trans_df = pd.DataFrame(transaction_history, columns=['action', 'market', 'price', 'size'])
    
    # Plot the profit over time
    plt.figure(figsize=(10,6))
    plt.plot(trans_df.index, trans_df['price'], label='Price', color='blue')
    plt.title('Arbitrage Strategy Performance')
    plt.xlabel('Time')
    plt.ylabel('Price')
    plt.legend()
    plt.show()

# Run the backtest
historical_data = fetch_historical_data()
final_balance, trans_history = backtest_arbitrage(historical_data)

# Print the final balance after backtesting
print(f"Final Balance: ${final_balance:.2f}")
plot_results(trans_history)